### COVID 19 cases, crime rate, and population data for Kansas, City MO
Data sources:

https://data.kcmo.org/Health/COVID-19-Data-by-ZIP-Code/98fz-2jyt

https://data.kcmo.org/Crime/KCPD-Crime-Data-2020/vsgj-uufz

https://github.com/datamade/census

https://public.opendatasoft.com/

https://data.kcmo.org/dataset/Zip-Codes/q2i4-9764 

In [18]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error

In [19]:
covid_file = 'Data/COVID-19_Data_by_ZIP_Code.csv'
covid_data = pd.read_csv(covid_file)

In [3]:
crime_file = 'Data/KCPD_Crime_Data_2020.csv'
crime_data = pd.read_csv(crime_file)

In [4]:
pop_file = 'Data/Population_by_Zipcode'
pop_data = pd.read_csv(pop_file)

In [5]:
crime_df = crime_data[['Report_No', "Zip Code"]]

In [6]:
crime_df = crime_df.rename(columns={'Zip Code': 'Zipcode'})

In [7]:
crime_df = crime_df.dropna(how='any')

In [8]:
crime_df['Zipcode'] = crime_df['Zipcode'].astype(int)

In [9]:
crime_df = crime_df.loc[crime_df['Zipcode'] > 64100]
crime_df = crime_df.loc[crime_df['Zipcode'] < 64168]
crime_df = crime_df.groupby('Zipcode').count()
crime_df

,Report_No
Zipcode,
64101,85
64102,45
64103,5
64104,4
64105,805
...,...
64161,208
64163,21
64164,21


In [10]:
pop_data = pop_data.loc[pop_data['city'] == "Kansas City"]
pop_df = pop_data.loc[pop_data['state'] == 'MO']
pop_df = pop_df.set_index('Zipcode')
pop_df['Population'] = pop_df['Population'].astype(int)
pop_df

,Population,city,state,latitude,longitude
Zipcode,,,,,
64167,446,Kansas City,MO,39.317300,-94.48655
64154,10895,Kansas City,MO,39.261679,-94.63353
64163,748,Kansas City,MO,39.318350,-94.67635
64166,285,Kansas City,MO,39.322849,-94.50755
64145,5249,Kansas City,MO,38.896407,-94.59666
64151,26777,Kansas City,MO,39.215652,-94.63021
64102,0,Kansas City,MO,39.093854,-94.60411
64109,9647,Kansas City,MO,39.065954,-94.56643
64125,1389,Kansas City,MO,39.104887,-94.49541


In [27]:
covid_df = covid_data[['ZipCode', 'COVID19 Cases']]
covid_df = covid_df.rename(columns={'ZipCode': 'Zipcode', 'COVID19 Cases': 'COVID19_Cases'})
covid_df = covid_df.set_index('Zipcode')
# covid_df.drop(covid_df['COVID19_Cases'] =='SUPP*', inplace=True)
# covid_df['COVID19_Cases'] = covid_df['COVID19_Cases'].astype(int)
covid_df

KeyError: "['SUPP*'] not found in axis"

In [12]:
engine = create_engine('sqlite:///covid_db.sqlite', echo=False)

In [14]:
covid_df.to_sql(name='covid_stats', con=engine, if_exists='replace', index=True)

In [15]:
pop_df.to_sql(name='pop_stats', con=engine, if_exists='replace', index=True)

In [16]:
crime_df.to_sql(name='crime_stats', con=engine, if_exists='replace', index=True)

In [17]:
engine.table_names()

['covid_stats', 'crime_stats', 'pop_stats']